# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import nltk
import pandas as pd
import re

from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.corpus.reader.wordnet import ADJ, ADV, NOUN, VERB
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\emq\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
with engine.connect() as connection:
    df = pd.read_sql_table("messages", connection)
df.head()

ValueError: Table messages not found

In [3]:
df['genre'].value_counts()

news      13053
direct    10766
social     2396
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [5]:
def convert_pos_tag(tag):
    # https://www.programcreek.com/python/example/91610/nltk.corpus.wordnet.NOUN
    if tag in ['JJ', 'JJR', 'JJS']:
        return ADJ
    elif tag in ['RB', 'RBR', 'RBS']:
        return ADV
    elif tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return NOUN
    elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return VERB
    return NOUN 

def tokenize(text):
    tokens = word_tokenize(text)
    words = [token for token in tokens if re.match("[a-zA-Z0-9]", token)]
    no_stopwords = [word for word in words if word not in stopwords.words("english")]
    lowercase_words = [word.lower() for word in no_stopwords]
    pos_tagged_words = pos_tag(lowercase_words)
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, pos=convert_pos_tag(pos)) for word, pos in pos_tagged_words]
    return lemmatized_words

print(tokenize(df.iloc[0, 0]))

NameError: name 'df' is not defined

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X = df['message']
y = df.drop(columns=['message', 'genre'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    "vect__ngram_range": [(1, 1), (1, 2)],
    "clf__estimator__n_estimators": [50, 100],
    "clf__estimator__max_features": ['sqrt', 'log2']}
cv = GridSearchCV(pipeline, parameters, n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.92      0.88      4947
           1       0.85      0.50      0.63      1076
           2       0.00      0.00      0.00        29
           3       0.78      0.63      0.70      2675
           4       0.73      0.06      0.11       538
           5       0.83      0.08      0.14       313
           6       0.81      0.08      0.14       167
           7       0.00      0.00      0.00       111
           8       0.75      0.04      0.08       223
           9       0.00      0.00      0.00         0
          10       0.89      0.46      0.61       420
          11       0.88      0.53      0.66       722
          12       0.83      0.34      0.48       566
          13       0.67      0.04      0.07       109
          14       0.57      0.03      0.05       151
          15       1.00      0.01      0.03        75
          16       0.60      0.01      0.03       209
          17       0.80    

C:\Users\emq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emq\anaconda3\lib\site-packages\sklearn\metrics\_classificat

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.